In [1]:
import sys
sys.path.append('/home/ec2-user/CpGPython/code/')
sys.path.append('/home/ec2-user/anaconda3/lib/python3.6/site-packages')
import pandas as pd
import numpy as np
from hyperopt import fmin,tpe,hp, STATUS_OK,Trials
import deep_network_estimator as dne
import Logger
from sklearn.model_selection import cross_val_score, cross_validate

/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
sys.path

['',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/anaconda3/envs/py35tf/lib/python3.5/site-packages/']

In [2]:
def loss_function(params):
    global estimators
    estimator = dne.tensor_DNN(home=home,**params)
    estimators.extend([estimator])
    score = cross_val_score(estimator,train_x,train_label,n_jobs=1).mean()
    return {'loss':-score,'status':STATUS_OK}

def best_estimator(trials,estimators):
    best_ix = np.argmin(trials.losses())
    return estimators[best_ix]

In [3]:
params = {'batch_normalization':True,'l2_reg':0.01,'n_classes':len(train_label.unique()),'hidden_layers':[int(train_num*1.5),int(train_num*1),int(train_num*0.5)],'drop_out':0.5,'weight_factor':1.5,'steps':2000,'batch_size':30,'scoring':'f1'}
estimator = dne.tensor_DNN(home=home,**params)
precision = cross_validate(estimator,train_x,train_label,cv=5,scoring=['precision','recall'],fit_params={'sample_weight':sample_weights_train})

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [4]:
precision

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([112.12593961, 120.59401941, 123.54858017, 113.14491153,
        110.34719372]),
 'score_time': array([8.29319334, 4.81801033, 7.98978615, 8.06820083, 8.21926808]),
 'test_precision': array([0.11111111, 0.        , 0.13043478, 0.09004739, 0.        ]),
 'train_precision': array([0.10508475, 0.14285714, 0.09763948, 0.09343584, 0.        ]),
 'test_recall': array([0.53125   , 0.        , 0.65217391, 1.        , 0.        ]),
 'train_recall': array([0.37804878, 0.06289308, 0.60666667, 1.        , 0.        ])}

In [2]:
home='/home/ec2-user/CpGPython/'
log_dir = home+'logs/'
logger = Logger.Logger(log_dir,False).get_logger()
with pd.HDFStore(home+'data/selected_features','r') as h5s:
    train_x =h5s['train_x'] 
    train_label = h5s['train_label'] 
    test_x = h5s['test_x'] 
    test_label = h5s['test_label']   
    sample_weights_train = h5s['sample_weights_train'] 
    sample_weights_test = h5s['sample_weights_test'] 
#train_x['sample_weights'] = sample_weights_train
#test_x['sample_weights'] = sample_weights_test
train_num = len(train_x)

In [5]:
space = {'batch_normalization': hp.choice('batch_normalization',[True]),
         'l2_reg': hp.uniform('l2_reg',0.001,0.05),
         'n_classes': hp.choice('n_classes',[len(train_label.unique())]),
         'hidden_layers': hp.choice('hidden_layers',[[int(train_num*1.5),int(train_num*1),int(train_num*0.5)],[int(train_num*1.6),int(train_num*1.2),int(train_num*0.8),int(train_num*0.4)],[int(train_num*1.5),int(train_num*1.2),int(train_num*0.9),int(train_num*0.6),int(train_num*0.3)],[int(train_num*2),int(train_num*1.5)]]),
         'drop_out':hp.uniform('drop_out',0.1,0.5),
         'weight_factor':hp.uniform('weight_factor',1,2),
         'steps':hp.choice('steps',np.linspace(200,2000,10,dtype='i8')),
         'batch_size':hp.choice('batch_size',[30]),
         'scoring':hp.choice('scoring',['precision']),
         }
estimators = []
trials = Trials()
best = fmin(loss_function,space,algo=tpe.suggest,max_evals=50,trials=trials)

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", s

ResourceExhaustedError: /home/ec2-user/CpGPython/tensor_logs/20180411220829/model.ckpt-1_temp_694070d22eda4f33b6bed67da3bf23c1/part-00000-of-00001.data-00000-of-00001.tempstate3726648215155998765; No space left on device
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/ShardedFilename, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, batch_normalization/beta, batch_normalization/beta/Adam, batch_normalization/beta/Adam_1, batch_normalization/gamma, batch_normalization/gamma/Adam, batch_normalization/gamma/Adam_1, batch_normalization/moving_mean, batch_normalization/moving_variance, batch_normalization_1/beta, batch_normalization_1/beta/Adam, batch_normalization_1/beta/Adam_1, batch_normalization_1/gamma, batch_normalization_1/gamma/Adam, batch_normalization_1/gamma/Adam_1, batch_normalization_1/moving_mean, batch_normalizati

In [7]:
print(best)

{'batch_normalization': 0, 'batch_size': 0, 'drop_out': 0.2126201171976041, 'hidden_layers': 2, 'l2_reg': 0.008698693257183789, 'n_classes': 0, 'scoring': 0, 'steps': 1, 'weight_factor': 1.8719054424366874}


In [14]:
np.argmin(trials.losses())

40